In [1]:
import cvxpy as cvx
import pandas as pd
import numpy as np

In [2]:
stocks = ['000001', '000002', '000003', '000004', '000005', '000006']
rets = pd.Series([-0.15, 0.25, 0.3, -0.1, -0.1, 0.2], index=stocks)

In [3]:
labels = {
    '000001': 'A',
    '000002': 'A',
    '000003': 'B',
    '000004': 'B',
    '000005': 'C',
    '000006': 'C',
}
min_weights = {'A': -0.2, 'B': -0.1, 'C': -0.4}
max_weights = {'A': 0.5, 'B': 0.1, 'C': 0.2}

In [4]:
labels = pd.Series(labels)
min_weights = pd.Series(min_weights)
max_weights = pd.Series(max_weights)

In [5]:
n = len(stocks)
assets = stocks

In [6]:
w = cvx.Variable(n)

In [7]:
alphas = rets.values
long_profit = alphas.T * long_w  # 多头加权收益
short_profit = alphas.T * short_w  # 空头加权收益
obj = cvx.Maximize(cvx.sum(long_profit + short_profit))

In [8]:
base_cons = []
if long_w:
    base_cons.append(long_w >= 0)
if short_w:
    base_cons.append(short_w <= 0)

In [9]:
group_a = long_w[0] + long_w[1] + short_w[0] + short_w[1]
group_b = long_w[2] + long_w[3] + short_w[2] + short_w[3]
group_c = long_w[4] + long_w[5] + short_w[4] + short_w[5]

In [10]:
constraints = []

In [11]:
for g, e in zip(['A','B','C'], (group_a,group_b,group_c)):
    constraints.extend([cvx.sum(e) >= min_weights[g],
                        cvx.sum(e) <= max_weights[g]])

In [12]:
constraints = constraints + base_cons + [cvx.sum(w) <= 2.0]

In [13]:
prob = cvx.Problem(obj, constraints)

In [14]:
prob.is_dcp()

True

In [15]:
prob.solve()

0.4352783398124776

In [16]:
prob.solver_stats.solver_name

'OSQP'

In [17]:
prob.status

'optimal'

In [18]:
np.round(long_w.value, 2)

array([0.  , 0.97, 0.3 , 0.  , 0.  , 0.06])

In [19]:
np.round(short_w.value, 2)

array([-0.47, -0.  , -0.  , -0.2 ,  0.  ,  0.  ])

In [20]:
np.round(w.value, 2)

array([0.47, 0.97, 0.3 , 0.2 , 0.  , 0.06])

In [21]:
long_profit = alphas.T * long_w.value  # 多头加权收益
short_profit = alphas.T * short_w.value  # 空头加权收益

In [22]:
sum(long_profit + short_profit)

0.4351840600624567

In [23]:
for i in range(0,6,2):
    print(sum(short_w.value[i:i+2] + long_w.value[i:i+2]))

0.5015360517960552
0.10064467768920698
0.06023509390946014


In [24]:
from zipline.optimize import NotConstrained

In [25]:
NotConstrained

'NotConstrained'

In [26]:
print(prob.status)
print('最优解', np.round(prob.value, 4))
print('绝对值权重\n',np.round(w.value, 4))
print('多头权重\n',np.round(l_w.value, 4))
print('空头权重\n',np.round(s_w.value, 4))

optimal
最优解 0.4353
绝对值权重
 [0.4713 0.9734 0.2985 0.1976 0.     0.0602]


NameError: name 'l_w' is not defined

In [ ]:
s = pd.Series([np.nan, 1,2])

In [ ]:
s.values